
# MODEL PROJECT
## Model Description

**Purpose**: This model describes the spread of an infectious disease and can be used to make important policy decisions when faced with an epidemic or pandemic, such as the covid-19 pandemic. Further, it can be used as a tool to see how diseases affect the behaviour of housholds regarding consumption. This model was also used by the "Økonomiske Råd" during covid when analyzing the effect of the pandemic on the economy. SIR models have recieved increased attention due to the pandemic. 

**Set up**: The population is divided into three different groups. 
- Susceptible ($S$), those who possible can get the disease
- Infectious ($I$), those who are infected by the disease and thus can spread it to other people
- Recovered ($R$), those who have gone through sickness and are assumed to be immune and cannot get disease again
- Population size ($N$) is equal to $N = S + I + R$. We assume that it is constant over time, even though there is also deaths due to the disease. 
- $\beta$ is the infection rate. Indicating the probability of becoming infected when you have been in contact with an infected person.
- $\gamma$ is the recovery rate, indicating how long it takes before the person is recoves and is immune against new infections. 
- $\mu$ is the mortality rate of the disease. 
- $dS_{v}/dt$ is the vaccination rate
- $V$ represents how many individuals are vaccinated. 

The model can be expressed by these equations: 

**Equations**: 
$$\begin{aligned}
\frac{dS}{dt} &= -\frac{\beta S I}{N} - \frac{dS_{v}}{dt} \\
\frac{dI}{dt} &= \frac{\beta S I}{N} - (\gamma + \mu) I \\
\frac{dR}{dt} &= \gamma I \\
\frac{dD}{dt} &= \mu I \\
\frac{dV}{dt} &= \frac{dS_{v}}{dt}
\end{aligned}$$

The first equation indicates how fast people get infected. The second represents the change in the number of infectious individuals, taking into account the infection rate and the recovery rate, and lastly also the mortality rate. The fourth equation represents the recovery rate. The fifth equation represents the vaccination rate. 

## Solving the model
### Without a Vaccination Program

As can be seen in the following graph, witha given description of the state of the world and of the virus, out model is able to illustrate how the infection will spread in the population before. As can be seen, the re will be a natural herd immunity, however, the difference between the two states comes more to show when comparing to the state with vaccines. 

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
%reload_ext autoreload
from modelproject import SIRModel3
import ipywidgets as widgets
from IPython.display import display

# Create an instance of the SIRModel3 class with the desired parameter values
model = SIRModel3(population=100000, infection_rate=0.4, recovery_rate=0.1, initial_infected=100, mortality_rate=0.015)


# Create interactive sliders for the model parameters
population_slider = widgets.IntSlider(value=model.N, min=1000, max=1000000, step=1000, description='Population:')
infection_rate_slider = widgets.FloatSlider(value=model.beta, min=0.1, max=2.0, step=0.1, description='Infection Rate:')
recovery_rate_slider = widgets.FloatSlider(value=model.gamma, min=0.01, max=0.5, step=0.01, description='Recovery Rate:')
initial_infected_slider = widgets.IntSlider(value=model.I0, min=10, max=1000, step=10, description='Initial Infected:')
mortality_rate_slider = widgets.FloatSlider(value=model.mu, min=0.001, max=0.1, step=0.001, description='Mortality Rate:')

# Function to update the plot based on the slider values
def update_plot(population, infection_rate, recovery_rate, initial_infected, mortality_rate):
    model = SIRModel3(population, infection_rate, recovery_rate, initial_infected, mortality_rate)
    model.plot_novax(num_days=100)

# Create an output widget for the plot
plot_output = widgets.Output()

# Define a function to handle widget changes
def handle_slider_change(change):
    plot_output.clear_output()
    with plot_output:
        update_plot(
            population=population_slider.value,
            infection_rate=infection_rate_slider.value,
            recovery_rate=recovery_rate_slider.value,
            initial_infected=initial_infected_slider.value,
            mortality_rate=mortality_rate_slider.value
        )

# Observe slider changes and update the plot accordingly
population_slider.observe(handle_slider_change, names='value')
infection_rate_slider.observe(handle_slider_change, names='value')
recovery_rate_slider.observe(handle_slider_change, names='value')
initial_infected_slider.observe(handle_slider_change, names='value')
mortality_rate_slider.observe(handle_slider_change, names='value')

# Display the sliders and plot output

display(population_slider, infection_rate_slider, recovery_rate_slider, initial_infected_slider, mortality_rate_slider)

display(plot_output)



IntSlider(value=100000, description='Population:', max=1000000, min=1000, step=1000)

FloatSlider(value=0.4, description='Infection Rate:', max=2.0, min=0.1)

FloatSlider(value=0.1, description='Recovery Rate:', max=0.5, min=0.01, step=0.01)

IntSlider(value=100, description='Initial Infected:', max=1000, min=10, step=10)

FloatSlider(value=0.015, description='Mortality Rate:', max=0.1, min=0.001, step=0.001)

Output()

### EXTENSION: Model with a Vaccine Program
When introducing the vaccine program we can see that we are able to 'flatten the curve'. All variables will decrease except mortality rate. This is explained by the simplicity of out model. It would be ideal and more realistic if the mortality rate was not given exogenously, but endogenously. 

In [8]:
# Create an instance of the SIRModel3 class with the desired parameter values
model = SIRModel3(population=100000, infection_rate=0.4, recovery_rate=0.1, initial_infected=100, mortality_rate=0.015, vaccination_rate=0.01)

# Create interactive sliders for the model parameters
population_slider = widgets.IntSlider(value=model.N, min=1000, max=1000000, step=1000, description='Population:')
infection_rate_slider = widgets.FloatSlider(value=model.beta, min=0.1, max=2.0, step=0.1, description='Infection Rate:')
recovery_rate_slider = widgets.FloatSlider(value=model.gamma, min=0.01, max=0.5, step=0.01, description='Recovery Rate:')
initial_infected_slider = widgets.IntSlider(value=model.I0, min=10, max=1000, step=10, description='Initial Infected:')
mortality_rate_slider = widgets.FloatSlider(value=model.mu, min=0.001, max=0.1, step=0.001, description='Mortality Rate:')
vaccination_rate_slider = widgets.FloatSlider(value=model.V0 / model.N, min=0.0, max=0.5, step=0.01, description='Vaccination Rate:')
display(population_slider, infection_rate_slider, recovery_rate_slider, initial_infected_slider, mortality_rate_slider, vaccination_rate_slider)

# Create an output widget for the plot
plot_output = widgets.Output()

# Function to update the plot based on the slider values
def update_plot(population, infection_rate, recovery_rate, initial_infected, mortality_rate, vaccination_rate):
    plot_output.clear_output()
    with plot_output:
        model = SIRModel3(population, infection_rate, recovery_rate, initial_infected, mortality_rate, vaccination_rate)
        model.plot(num_days=100, vaccination_rate=vaccination_rate)

# Define a function to handle slider changes
def handle_slider_change(change):
    update_plot(
        population=population_slider.value,
        infection_rate=infection_rate_slider.value,
        recovery_rate=recovery_rate_slider.value,
        initial_infected=initial_infected_slider.value,
        mortality_rate=mortality_rate_slider.value,
        vaccination_rate=vaccination_rate_slider.value
    )

# Observe slider changes and update the plot accordingly
population_slider.observe(handle_slider_change, names='value')
infection_rate_slider.observe(handle_slider_change, names='value')
recovery_rate_slider.observe(handle_slider_change, names='value')
initial_infected_slider.observe(handle_slider_change, names='value')
mortality_rate_slider.observe(handle_slider_change, names='value')
vaccination_rate_slider.observe(handle_slider_change, names='value')

# Display the plot output
display(plot_output)


IntSlider(value=100000, description='Population:', max=1000000, min=1000, step=1000)

FloatSlider(value=0.4, description='Infection Rate:', max=2.0, min=0.1)

FloatSlider(value=0.1, description='Recovery Rate:', max=0.5, min=0.01, step=0.01)

IntSlider(value=100, description='Initial Infected:', max=1000, min=10, step=10)

FloatSlider(value=0.015, description='Mortality Rate:', max=0.1, min=0.001, step=0.001)

FloatSlider(value=0.01, description='Vaccination Rate:', max=0.5, step=0.01)

Output()

### Optimization of Vaccination rate to minimize deaths. 
The code find the vaccination rate that minimizes the mortality. The optimal vaccination rate that was obtained from the 'optimize_vaccination_rate` method is 0.999, which means that vaccinating almost the entire population is the best way to minimize the total number of deaths over the 100-day period. This is not surprising, as vaccination is one of the most effective measures to prevent the spread of infectious diseases.

The total number of deaths with the optimal vaccination rate is 18.25, which is a relatively small number compared to the size of the population (1,000,000) and the initial number of infected people (100). This is a testament to the effectiveness of the vaccination strategy in controlling the spread of the disease.

It is worth noting that the model assumes a constant vaccination rate over the 200-day period. In reality, the vaccination rate may vary over time due to factors such as vaccine availability, public perception, and government policies. Therefore, the optimal vaccination rate obtained from the model should be taken as a guideline rather than a definitive answer.

By adding more variables to the model, the optimal vaccination rate would become more realistic. Also, the cost of the vaccine could have been included in order to measure the effect of the vaccine vs. the cost of it. 

In [9]:
optimal_vaccination_rate = model.optimize_vaccination_rate(num_days=100)
print("Optimal vaccination rate:", optimal_vaccination_rate.x)
print("Total deaths with optimal vaccination rate:", model.total_deaths(num_days=100, vaccination_rate=optimal_vaccination_rate.x))

Optimal vaccination rate: 0.9999939459326207
Total deaths with optimal vaccination rate: 18.252909861007275


## Conclusion 
In this project we have made a SIR-model that models the spread of an infectious disease wihtin a population. We extended the model to include vaccines and found the optimal vaccination rate. A possible extension to the model would be to add more variables in order for the model to be more precise or to model a shock where the disease got either more deadly or more infectious. 
